In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
os.listdir('./Dataset/')

['training', 'validation']

In [3]:
train_path = r'Dataset\training'
test_path = r'Dataset\validation'

In [4]:
img_size = 224
# normalization parameters
mean = [0.2842, 0.3798, 0.4523]
std  = [0.2231, 0.1942, 0.1880]

#Applying Transformation
train_transforms = transforms.Compose([transforms.Resize((img_size,img_size)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(10),
                                transforms.ToTensor(),
                                transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))])

test_transforms = transforms.Compose([transforms.Resize((img_size,img_size)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))])

In [5]:
train_dataset = torchvision.datasets.ImageFolder(root = train_path, 
                                                 transform=train_transforms)
test_dataset = torchvision.datasets.ImageFolder(root = test_path, 
                                                 transform=test_transforms)

In [6]:
BATCH_SIZE = 32

torch.manual_seed(0)
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           shuffle=True, drop_last=True, num_workers=4)
torch.manual_seed(0)
test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=BATCH_SIZE, 
                                          shuffle=False, num_workers=4)
len(train_loader), len(test_loader)

(24, 3)

In [7]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

resnet18_model = models.resnet18(weights = True)
classes = 2 # five categories
num_ftrs = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_ftrs, classes)
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(resnet18_model.parameters(), 
                      lr = 0.005, 
                      momentum = 0.9, 
                      weight_decay=0.003 # extra error to prevent over-fitting
                     )

c:\Users\anshu\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
def save_checkpoint(model, epoch, optimizer, best_acc):
    state = {
        'epoch': epoch + 1,
        'model': model.state_dict(),
        'best accuracy': best_acc,
        'optimizer': optimizer.state_dict(),
        'comments': 'Noice!!',
    }
    torch.save(state, 'model_best_checkpoint.pth.tar')

In [9]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
    best_acc = 0
    
    for epoch in range(n_epochs):
        print("Epoch number %d " % (epoch + 1))
        model.train()
        running_loss = 0.0 
        running_correct = 0.0
        total = 0
        
        for data in train_loader:
            images, labels = data
            total += labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss = loss.item()
            running_correct += (labels==predicted).sum().item()
        
        epoch_loss = running_loss/len(train_loader)
        epoch_acc = 100.00 * running_correct / total
        
        print("- Training dataset. Got %d out of %d images correctly (%.3f%%). Epoch Loss: %.3f"
              % (running_correct, total, epoch_acc, epoch_loss))
        
        test_dataset_acc = evaluate_model_test(model, test_loader)
    
        if(test_dataset_acc > best_acc):
            best_acc = test_dataset_acc
            save_checkpoint(model, epoch, optimizer, best_acc)
        
    print("Finished")
    return model

In [10]:
def evaluate_model_test(model, test_loader):
    model.eval()
    predict_correct = 0
    total = 0
    
    with torch.inference_mode():
        for data in test_loader:
            images, labels = data
            total += labels.size(0)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            predict_correct += (labels==predicted).sum().item()
    epoch_acc = 100.00 *predict_correct / total
    
    print("- Testing dataset. Got %d out of %d images correctly (%.3f%%)"
              % (predict_correct, total, epoch_acc))
    
    return epoch_acc

In [11]:
train_nn(resnet18_model, train_loader, test_loader, loss_fn, optimizer, 10)

Epoch number 1 
- Training dataset. Got 725 out of 768 images correctly (94.401%). Epoch Loss: 0.001
- Testing dataset. Got 69 out of 70 images correctly (98.571%)
Epoch number 2 
- Training dataset. Got 768 out of 768 images correctly (100.000%). Epoch Loss: 0.000
- Testing dataset. Got 70 out of 70 images correctly (100.000%)
Epoch number 3 
- Training dataset. Got 768 out of 768 images correctly (100.000%). Epoch Loss: 0.000
- Testing dataset. Got 70 out of 70 images correctly (100.000%)
Epoch number 4 
- Training dataset. Got 768 out of 768 images correctly (100.000%). Epoch Loss: 0.000
- Testing dataset. Got 70 out of 70 images correctly (100.000%)
Epoch number 5 
- Training dataset. Got 768 out of 768 images correctly (100.000%). Epoch Loss: 0.000
- Testing dataset. Got 70 out of 70 images correctly (100.000%)
Epoch number 6 
- Training dataset. Got 768 out of 768 images correctly (100.000%). Epoch Loss: 0.000
- Testing dataset. Got 70 out of 70 images correctly (100.000%)
Epoch 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
checkpoint = torch.load('model_best_checkpoint.pth.tar')

In [13]:
print(checkpoint['epoch'])
print(checkpoint['comments'])
print(checkpoint['best accuracy'])

2
Noice!!
100.0


In [15]:
resnet18_model = models.resnet18()
num_ftrs = resnet18_model.fc.in_features
classes = 2
resnet18_model.fc = nn.Linear(num_ftrs, classes)
resnet18_model.load_state_dict(checkpoint['model'])

torch.save(resnet18_model, 'best_model.pth')